<a href="https://colab.research.google.com/github/Ruheena-S/Hierarchical-classification-Loss-Functions-in-Image-Classification/blob/main/OVA_ResNet50_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch_optimizer torchmetrics
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 46.8 MB/s eta 0:00:00
Mon Mar 13 22:47:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    25W /  70W |      0MiB / 15360MiB |      0%    

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_optimizer import Ranger
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms,models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchmetrics import Accuracy
from torch.optim.lr_scheduler import CyclicLR

torch.manual_seed(42)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:

def target_trans(target):
  y = -torch.ones(100)
  y[target] = 1

  return y,target

In [6]:
# Define transforms for the dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [7]:
dataset = datasets.CIFAR100(root='data/', train=True, download=True, transform=transform_train,target_transform=target_trans)
test_dataset = datasets.CIFAR100(root='data/', train=False, transform=transform_test,target_transform=target_trans)


val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
batch_size=256

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(val_ds, batch_size, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size, num_workers=4)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data/


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
class Hinge_Loss(torch.nn.Module):
    
    def __init__(self):
        super(Hinge_Loss,self).__init__()
        
    def forward(self,y_p,y_t):

        temp = 1 - y_p * y_t
        
        zero = torch.zeros(y_p.size()[0],y_p.size()[1])
        zero = zero.cuda()
        clamp = torch.max(temp,zero)
    
        total_loss = torch.sum(clamp)/y_p.size()[0]
        return total_loss

In [9]:
num_classes = 100
num_epochs = 50

In [11]:
# Define the ResNet50 model and freeze layers

model = models.resnet50(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size = (3,3), padding = (1, 1), bias = False)
model.maxpool = nn.Identity()

for param in model.parameters():
    param.requires_grad = True
for param in model.layer4.parameters():
    param.requires_grad = True

model.fc = nn.Linear(2048, 100)

model.to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

In [12]:
# Define loss function and optimizer

criterion = Hinge_Loss().to(device)
accuracy = Accuracy(task="multiclass",num_classes=num_classes).to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = Ranger(model.parameters(), lr=1e-3, weight_decay=1e-4) 
scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-3, step_size_up=len(train_loader)//2, cycle_momentum=False)

In [13]:
# Train the model

total_train_step = len(train_loader)
#print(total_train_step)
total_val_step=len(valid_loader)
BEST_VAL_METRIC = 0
BEST_MODEL = None

for epoch in range(1, num_epochs+1):

    train_loss=0
    train_acc=0.0
    model.train()

    for i, (images, target) in enumerate(train_loader, 1):

        y_trans = target[0]
        y_true = target[1]

        # Move tensors to the configured device
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, y_trans)

        train_loss += loss
        #train_acc += get_accuracy(outputs, y_true)
        train_acc += accuracy(outputs, y_true)
        
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    print(f'Epoch [{epoch}/{num_epochs}] - Loss: {(train_loss/total_train_step):.4f}, Accuracy: {(train_acc/total_train_step):.4f}')

    model.eval() 
    # Validation
    with torch.no_grad():
        val_acc = 0
        val_loss=0
        for i, (images, target) in enumerate(valid_loader, 1):

            y_trans = target[0]
            y_true = target[1]

            # Move tensors to the configured device
            images = images.to(device)
            y_true = y_true.to(device)
            y_trans = y_trans.to(device)

            outputs = model(images)
            val_loss += criterion(outputs, y_trans)
            #val_acc += get_accuracy(outputs, y_true)
            val_acc += accuracy(outputs, y_true)

    if val_acc/total_val_step > BEST_VAL_METRIC:
        BEST_VAL_METRIC = val_acc/total_val_step
        BEST_MODEL = model.state_dict() 
        torch.save(model, "/content/drive/MyDrive/MTP_Phase2/saved_models/resnet50_OVA_cifar100_RangerOPT.pth")

    print(f'Accuracy of the network on the 5000 validation images: {(val_acc/total_val_step):.4f}, loss: {(val_loss/total_val_step):.4f}') 

/usr/local/lib/python3.9/dist-packages/pytorch_ranger/ranger.py:172: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


Epoch [1/50] - Loss: 22.8888, Accuracy: 0.0289
Accuracy of the network on the 5000 validation images: 0.0446, loss: 2.6125
Epoch [2/50] - Loss: 2.2911, Accuracy: 0.0586
Accuracy of the network on the 5000 validation images: 0.0907, loss: 2.0837
Epoch [3/50] - Loss: 2.1239, Accuracy: 0.1230
Accuracy of the network on the 5000 validation images: 0.1517, loss: 2.0591
Epoch [4/50] - Loss: 2.0675, Accuracy: 0.1782
Accuracy of the network on the 5000 validation images: 0.2314, loss: 2.0312
Epoch [5/50] - Loss: 2.0588, Accuracy: 0.2333
Accuracy of the network on the 5000 validation images: 0.2831, loss: 2.0139
Epoch [6/50] - Loss: 2.0030, Accuracy: 0.2828
Accuracy of the network on the 5000 validation images: 0.3313, loss: 1.9586
Epoch [7/50] - Loss: 1.9605, Accuracy: 0.3312
Accuracy of the network on the 5000 validation images: 0.3730, loss: 1.9242
Epoch [8/50] - Loss: 1.9134, Accuracy: 0.3678
Accuracy of the network on the 5000 validation images: 0.4196, loss: 1.8529
Epoch [9/50] - Loss: 1.

In [14]:
#Testing

model.load_state_dict(BEST_MODEL)

total_test_step=len(test_loader)

with torch.no_grad():
    test_acc=0
    test_loss=0

    for i, (images, target) in enumerate(test_loader, 1):
        
        y_trans = target[0]
        y_true = target[1]
        
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)

        # Forward pass
        outputs = model(images)
        
        # Loss
        test_loss += criterion(outputs,y_trans)
        test_acc += accuracy(outputs, y_true)

    print(f'Accuracy of the network on test images: {(test_acc/total_test_step):.4f}, loss: {(test_loss/total_test_step):.4f}')

Accuracy of the network on test images: 0.7587, loss: 1.1303
